### Overview

In this notebook, we will collect data to explore how well-being metrics relate to overall metrics and analyze the evolution of mental health recognition over the years, focusing on specific companies and job roles.

Sample: 100 000 reviews
- Current Employee: 53730
- Former Employee: 46270

### Import libraries with neccessary functions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import re
import spacy
from collections import Counter
from textblob import TextBlob
from tqdm.notebook import tqdm

### Import initial dataset from Kaggle

In [2]:
reviews = pd.read_csv("../data/raw/all_reviews.csv", low_memory=False)

### Cleaning functions

In [3]:
# Function to clean numerical rating columns
def clean_rating_columns(df, columns):
    """
    Clean numerical rating columns by converting them to float and removing invalid entries.
    """
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df.dropna(subset=[col])
        df[col] = df[col].astype(float)
    return df

# Function to handle missing values
def handle_missing_values(df, columns_to_fill, fill_value=0.0):
    """
    Fill missing values for specified columns with a given value.
    """
    for col in columns_to_fill:
        df[col] = df[col].fillna(fill_value).astype(float)
    return df

# Function to extract year from date
def extract_year(df, date_column, year_column):
    """
    Extract the year from a date column and create a new year column.
    """
    df[year_column] = df[date_column].str.split().str[2].astype(int)
    df.drop(columns = "date", inplace = True)
    return df[df[year_column] > 2014]

# Function to extract firm names from the firm_link
def extract_firm_names(df, firm_link_column, firm_name_column):
    """
    Extract firm names from firm_link and filter firms with frequent mentions.
    """
    def extract_name(firm_link):
        match = re.search(r'(?<=/)([A-Za-z0-9\-]+)(?=-Reviews)', firm_link)
        return match.group(1) if match else None

    df[firm_name_column] = df[firm_link_column].apply(extract_name)
    df.drop(columns = "firm_link", inplace=True)
    firm_counts = df[firm_name_column].value_counts()
    valid_firms = firm_counts[firm_counts >= 100].index
    
    return df[df[firm_name_column].isin(valid_firms)]

# Function to unify employment statuses
def unify_employment_statuses(df, status_column):
    """
    Standardize employment statuses.
    """
    def unify_status(status):
        if "Current" in status:
            if "Contractor" in status:
                return "Current Contractor"
            elif "Freelancer" in status:
                return "Current Freelancer"
            elif "Intern" in status:
                return "Current Intern"
            elif "Self-employed" in status:
                return "Current Self-employed"
            else:
                return "Current Employee"
        elif "Former" in status:
            if "Contractor" in status:
                return "Former Contractor"
            elif "Freelancer" in status:
                return "Former Freelancer"
            elif "Intern" in status:
                return "Former Intern"
            elif "PhD" in status:
                return "Former PhD"
            elif "Self-employed" in status:
                return "Former Self-employed"
            else:
                return "Former Employee"
        else:
            return "Other"

    df[status_column] = df[status_column].apply(unify_status)
    return df

# Main cleaning function
def clean_dataset(file_path, rating_columns, missing_columns, firm_link_column, status_column, date_column, year_column, job_column):
    """
    Clean the dataset by applying all cleaning functions step by step.
    """
    # Read the data
    df = pd.read_csv(file_path, low_memory=False)
    
    # Clean rating columns
    df = clean_rating_columns(df, rating_columns)
    
    # Drop unnecessary columns
    df.drop(columns=["title", "advice", "index", "Business Outlook"], inplace=True)
    
    # Drop rows with significant missing values
    df.dropna(subset=['pros', 'cons', 'Culture & Values'], inplace=True)

    #Unify names of the columns and reset index
    df['pros'] = df['pros'].str.lower().str.replace(r'\n', ' ', regex=True)
    df['cons'] = df['cons'].str.lower().str.replace(r'\n', ' ', regex=True)
    
    
    # Extract year and filter rows
    df = extract_year(df, date_column, year_column)
    
    # Handle missing values in specific columns
    df = handle_missing_values(df, missing_columns, fill_value=0.0)
    
    # Extract and filter firm names
    df = extract_firm_names(df, firm_link_column, 'firm_name')

    # Unify employment statuses
    df = unify_employment_statuses(df, status_column)
    
    return df

# Usage
file_path = "../data/raw/all_reviews.csv"
rating_columns = ['Work/Life Balance', 'Compensation and Benefits', 'Senior Management', 'Career Opportunities']
missing_columns = ['Diversity & Inclusion']
firm_link_column = 'firm_link'
status_column = 'status'
date_column = 'date'
year_column = 'year'
job_column = 'job'

cleaned_reviews = clean_dataset(
    file_path=file_path,
    rating_columns=rating_columns,
    missing_columns=missing_columns,
    firm_link_column=firm_link_column,
    status_column=status_column,
    date_column=date_column,
    year_column=year_column,
    job_column=job_column
)


/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/4252623836.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)
/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/4252623836.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')


### Unify column names

In [4]:
#lowercase column names and replace space by "_"
cleaned_reviews.columns = cleaned_reviews.columns.str.lower().str.replace(" ", "_", regex=True).str.replace("&", "and", regex=True)
cleaned_reviews.reset_index(drop=True, inplace=True)
print(cleaned_reviews.shape)
display(cleaned_reviews.head())

(6016214, 15)


,rating,status,pros,cons,recommend,ceo_approval,career_opportunities,compensation_and_benefits,senior_management,work/life_balance,culture_and_values,diversity_and_inclusion,job,year,firm_name
0,5.0,Current Employee,great life canada i liked,my dream canada working and enjoy the life,v,v,4.0,3.0,4.0,5.0,3.0,4.0,Delivery Post Person With Driving,2023,Canada-Life
1,5.0,Current Employee,best place to work always,system is a bit old,v,v,4.0,4.0,4.0,4.0,4.0,4.0,Manager,2023,Canada-Life
2,1.0,Former Employee,the pro is you get paid... i guess. i worked o...,"just felt like another cog in the machine, wor...",v,v,1.0,1.0,1.0,1.0,1.0,1.0,Customer Service Specialist,2022,Canada-Life
3,1.0,Former Employee,my peers were good to work with. everyone will...,"leadership is not helpful at all, they walk ar...",x,x,1.0,3.0,1.0,2.0,2.0,2.0,Anonymous Employee,2022,Canada-Life
4,4.0,Current Employee,great team with strong focus on personal devel...,company size often results in slow executive d...,v,v,4.0,3.0,4.0,4.0,5.0,5.0,Senior Workplace Strategist,2022,Canada-Life


### Cleaning job titles

In [5]:
def clean_job_titles(df, job_column):
    """
    Clean and standardize job titles in a DataFrame.

    """
    # Replace empty strings or NaN values with "Anonymous Employee"
    df[job_column].replace(' ', 'Anonymous Employee', inplace=True)
    df[job_column].fillna('Anonymous Employee', inplace=True)
    
    # Strip leading/trailing spaces
    df[job_column] = df[job_column].str.strip()
    
    # Count occurrences of each job title
    job_counts = df[job_column].value_counts()
    
    # Filter out jobs mentioned less than 100 times
    valid_jobs = job_counts[job_counts >= 100].index
    df = df[df[job_column].isin(valid_jobs)]
    
    # Replace specific job titles
    job_replacements = {
        'Salees Associate': 'Sales Associate', 'Sales Executive': 'Sales Associate',
        'IT Strategy Consultant': 'IT Consultant',
        'Sales Associate/Cashier': 'Cashier', 'Front End Cashier': 'Cashier',
        'Senior Software Engineer': 'Software Engineer', 
        'Senior Systems Engineer': 'Software Engineer', 
        'Senior Software Developer': 'Software Engineer', 
        'Associate Software Engineer': 'Software Engineer',
        'Sales': 'Sales Associate', 
        'Salees Associate ': 'Sales Associate', 
        'Retail Sales Associate': 'Sales Associate',
        'Sales Assistant': 'Sales Associate', 
        'Sales Representative': 'Sales Associate', 'Inside Sales Representative': 'Sales Associate',
        'Systems Engineer': 'Software Engineer', 'Senior Engineer':'Software Engineer',
        'Senior Associate': 'Associate', 
        'Senior Analyst ': 'Analyst',
        'Business Analyst': 'Analyst', 'Senior Business Analyst ': 'Analyst','Senior Business Analyst': 'Analyst', 'Financial Analyst': 'Analyst',
        'IT Analyst': 'Analyst', 'Senior Analyst': 'Analyst', 'Programmer Analyst': 'Analyst',
        'Server': 'Waiter/Waitress', 'Waiter': 'Waiter/Waitress', 'Waitress': 'Waiter/Waitress',
        'Warehouse Worker': 'Warehouse Associate',
        'Human Resources': 'HR', 
        'Recruiter': 'HR',
        'Customer Service Representative': 'Customer Service', 
        'Customer Service Representative (CSR)': 'Customer Service', 'Customer Service Specialist': 'Customer Service',
        'Customer Assistant': 'Customer Service', 
        'Customer Service Associate': 'Customer Service',
        'Executive Assistant': 'Administrative Assistant', 
        'Associate Director': 'Director',
        'Shift Supervisor': 'Supervisor',
        'Graduate Research Assistant': 'Researcher','Research Assistant': 'Researcher',
    }
    df[job_column] = df[job_column].replace(job_replacements)
    
    # Apply broader job title standardizations
    def standardize_jobs(job):
        if 'Teacher' in job:
            return 'Teacher'
        elif 'Manager' in job:
            return 'Manager'
        elif 'Consultant' in job:
            return 'Consultant'
        elif 'Teller' in job:  
            return 'Bank Teller'
        elif 'Intern' in job:
            return 'Intern'
        elif 'Pharmacy' in job:
            return 'Pharmacy'
        elif 'Team Member' in job or 'Crew Member' in job:
            return 'Team Member'
        return job
    
    df[job_column] = df[job_column].apply(standardize_jobs)
    
    return df

cleaned_reviews = clean_job_titles(cleaned_reviews, 'job') #apply the job cleaning function

/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/2627540091.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[job_column] = df[job_column].replace(job_replacements)
/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/2627540091.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[job_column] = df[job_column].apply(standardize_jobs)


### Getting a sample of the dataset for further analysis

In [6]:
sampled_reviews = cleaned_reviews.head(100000)
print(sampled_reviews["rating"].value_counts())
print(cleaned_reviews["rating"].value_counts())


print(sampled_reviews["work/life_balance"].value_counts())
print(cleaned_reviews["work/life_balance"].value_counts())

print(sampled_reviews["work/life_balance"].value_counts())
print(cleaned_reviews["work/life_balance"].value_counts())


rating
4.0    28697
5.0    25063
3.0    23832
2.0    11522
1.0    10886
Name: count, dtype: int64
rating
4.0    1432242
5.0    1345360
3.0    1094993
2.0     500317
1.0     475828
Name: count, dtype: int64
work/life_balance
5.0    24176
3.0    23504
4.0    23310
1.0    15776
2.0    13234
Name: count, dtype: int64
work/life_balance
5.0    1299540
4.0    1180749
3.0    1103259
1.0     675711
2.0     589481
Name: count, dtype: int64
work/life_balance
5.0    24176
3.0    23504
4.0    23310
1.0    15776
2.0    13234
Name: count, dtype: int64
work/life_balance
5.0    1299540
4.0    1180749
3.0    1103259
1.0     675711
2.0     589481
Name: count, dtype: int64


### Getting emotional status

In [7]:
# Function to flag stress, burnout, and anxiety
def emotional_status(text):
    if not isinstance(text, str):  # Check if text is not None or NaN
        return False, False, False, False, False, False  # Return False for all flags if the text is invalid

    text = text.lower()

    # Keywords for flags
    stress_keywords = ['stress', 'overwhelmed', 'nervous', 'pressure', 'stressful', 'deadlines', 'high demand', 'high workload']
    toxic_keywords = ['toxic','negative', 'micromanagement', 'lack of support', 'hostile', 'bullying', 'discrimination']
    burnout_keywords = ['burnout', 'overtasked', 'overworked', 'burned out', 'burning out', 'exhaustion', 'fatigue']
    anxiety_keywords = ['anxious', 'anxiety', 'panic', 'workload stress', 'job insecurity', 'sleepless']
    depression_keywords= ['depressed','sad','no motivation','unmotivated', 'headaches', 'feeling worthless']
    supportive_keywords = ['supportive', 'understanding', 'encouraging', 'teamwork']    
    

    # Remove unwanted phrases
    unwanted_phrases = ['not too stressful', 'no negatives']
    if any(phrase in text for phrase in unwanted_phrases):
        return False, False, False, False, False

    # Check for each set of keywords
    stress_flag = any(keyword in text for keyword in stress_keywords)
    burnout_flag = any(keyword in text for keyword in burnout_keywords)
    anxiety_flag = any(keyword in text for keyword in anxiety_keywords)
    toxic_flag = any(keyword in text for keyword in toxic_keywords)
    supportive_env_flag = any(keyword in text for keyword in supportive_keywords)
    depression_flag = any(keyword in text for keyword in depression_keywords)
    
    return stress_flag, burnout_flag, anxiety_flag, toxic_flag, supportive_env_flag, depression_flag




In [8]:
# Apply the emotional_status function to "pros" and "cons"
sampled_reviews[['stress_flag', 'burnout_flag', 'anxiety_flag', 'toxic_flag', 'supportive_env_flag', 'depression_flag']] = pd.DataFrame(
    sampled_reviews['pros'].apply(emotional_status).tolist(), index=sampled_reviews.index)

sampled_reviews[['stress_flag_cons', 'burnout_flag_cons', 'anxiety_flag_cons', 'toxic_flag_cons', 'supportive_env_flag_cons', 'depression_flag_cons']] = pd.DataFrame(
    sampled_reviews['cons'].apply(emotional_status).tolist(), index=sampled_reviews.index)

/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/1400904462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_reviews[['stress_flag', 'burnout_flag', 'anxiety_flag', 'toxic_flag', 'supportive_env_flag', 'depression_flag']] = pd.DataFrame(
/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/1400904462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_reviews[['stress_flag', 'burnout_flag', 'anxiety_flag', 'toxic_flag', 'supportive_env_flag', 'depression_flag']] 

### Check the median values for emotional flags & rating correlation

In [9]:
# List of emotional flags and their corresponding column names
emotional_flags = [
    ('burnout_flag', 'burnout_flag_cons'),
    ('stress_flag', 'stress_flag_cons'),
    ('anxiety_flag', 'anxiety_flag_cons'),
    ('toxic_flag', 'toxic_flag_cons'),
    ('supportive_env_flag', 'supportive_env_flag_cons'),
    ('depression_flag', 'depression_flag_cons')
]

# Loop through each flag and compute statistics
for flag, cons_flag in emotional_flags:
    filtered_df = sampled_reviews[(sampled_reviews[flag] == True) | (sampled_reviews[cons_flag] == True)]
    print(f"Results for {flag}:")
    print(f"Shape: {filtered_df.shape}")
    print(f"Median Rating: {filtered_df['rating'].median()}")
    print(f"Median Work/Life Balance: {filtered_df['work/life_balance'].median()}")
    print("-" * 40)

Results for burnout_flag:
Shape: (947, 27)
Median Rating: 3.0
Median Work/Life Balance: 2.0
----------------------------------------
Results for stress_flag:
Shape: (5798, 27)
Median Rating: 3.0
Median Work/Life Balance: 3.0
----------------------------------------
Results for anxiety_flag:
Shape: (167, 27)
Median Rating: 2.0
Median Work/Life Balance: 2.0
----------------------------------------
Results for toxic_flag:
Shape: (2772, 27)
Median Rating: 2.0
Median Work/Life Balance: 2.0
----------------------------------------
Results for supportive_env_flag:
Shape: (3114, 27)
Median Rating: 4.0
Median Work/Life Balance: 4.0
----------------------------------------
Results for depression_flag:
Shape: (396, 27)
Median Rating: 2.0
Median Work/Life Balance: 3.0
----------------------------------------


In [10]:
sampled_reviews
sampled_reviews.to_csv("../data/clean/sampled_reviews.csv", encoding="utf-8-sig", header=True)

,rating,status,pros,cons,recommend,ceo_approval,career_opportunities,compensation_and_benefits,senior_management,work/life_balance,...,anxiety_flag,toxic_flag,supportive_env_flag,depression_flag,stress_flag_cons,burnout_flag_cons,anxiety_flag_cons,toxic_flag_cons,supportive_env_flag_cons,depression_flag_cons
1,5.0,Current Employee,best place to work always,system is a bit old,v,v,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,False,False
2,1.0,Former Employee,the pro is you get paid... i guess. i worked o...,"just felt like another cog in the machine, wor...",v,v,1.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
3,1.0,Former Employee,my peers were good to work with. everyone will...,"leadership is not helpful at all, they walk ar...",x,x,1.0,3.0,1.0,2.0,...,False,False,False,False,False,False,False,False,False,False
5,4.0,Current Employee,good salary\r inclusive environment\r caring f...,i don't have any cons.,v,v,4.0,4.0,4.0,4.0,...,False,False,False,False,False,False,False,False,False,False
6,1.0,Former Employee,co-workers and close to public tramsit,high caseload\r incompetent management \r unre...,x,x,1.0,1.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121448,3.0,Former Employee,"global company, lots of opportunities, continu...",large company means it's easy to fall through ...,v,v,5.0,2.0,5.0,5.0,...,False,False,True,False,False,False,False,False,False,False
121449,4.0,Current Employee,good work culture and provides hands on with n...,not good growth in compensation and promotions,v,v,4.0,3.0,4.0,4.0,...,False,False,False,False,False,False,False,False,False,False
121450,5.0,Current Employee,"great work culture, care about their people","so far, just the pandemic and the distance",o,v,5.0,3.0,5.0,5.0,...,False,False,False,False,False,False,False,False,False,False
121451,3.0,Former Employee,- good work life balance\r - stronger control ...,- everything is project dependent. good client...,o,o,3.0,3.0,4.0,4.0,...,False,False,False,False,False,False,False,False,False,False


### Sentiment analysis

In [11]:
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

# Apply the sentiment analysis function to 'pros' and 'cons'
sampled_reviews[['pros_sentiment', 'pros_subjectivity']] = sampled_reviews['pros'].apply(lambda x: pd.Series(get_sentiment(x)))
sampled_reviews[['cons_sentiment', 'cons_subjectivity']] = sampled_reviews['cons'].apply(lambda x: pd.Series(get_sentiment(x)))

/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/1633712011.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_reviews[['pros_sentiment', 'pros_subjectivity']] = sampled_reviews['pros'].apply(lambda x: pd.Series(get_sentiment(x)))
/var/folders/7h/j883bpy50yx9ffn06wrdxzy00000gn/T/ipykernel_1345/1633712011.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_reviews[['pros_sentiment', 'pros_subjectivity']] = sampled_reviews['pros'].apply(lambda x: pd.Series(get_sentiment(x)))
/v

In [12]:
pd.set_option('display.max_columns', None)
nlp = spacy.load('en_core_web_sm')
sampled_reviews.columns

Index(['rating', 'status', 'pros', 'cons', 'recommend', 'ceo_approval',
       'career_opportunities', 'compensation_and_benefits',
       'senior_management', 'work/life_balance', 'culture_and_values',
       'diversity_and_inclusion', 'job', 'year', 'firm_name', 'stress_flag',
       'burnout_flag', 'anxiety_flag', 'toxic_flag', 'supportive_env_flag',
       'depression_flag', 'stress_flag_cons', 'burnout_flag_cons',
       'anxiety_flag_cons', 'toxic_flag_cons', 'supportive_env_flag_cons',
       'depression_flag_cons', 'pros_sentiment', 'pros_subjectivity',
       'cons_sentiment', 'cons_subjectivity'],
      dtype='object')

### Load spacy library for natural language procesing & get the frequency counts per rating

In [13]:
nlp = spacy.load('en_core_web_sm')


# Use your preprocessing function
def preprocess_text_spacy(text):
    # Parse the text using SpaCy
    doc = nlp(text.lower())
    
    # Filter tokens: keep only alphabetic words, exclude stopwords and punctuation
    """
    #token.is_alpha: This condition filters out any tokens that are not alphabetic (e.g., numbers, punctuation)
    not token.is_stop: This excludes common stopwords (e.g., "the", "a", "and") 
    token.lemma_: converts each word to its base form (lemma). For example, "running" becomes "run", and "better" becomes "good".
    """
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    
    #  after filtering and lemmatizing, the individual tokens (words) are joined back into a single string with spaces separating them.
    processed_text = ' '.join(tokens)
    
    return processed_text

# Function to get word frequencies from text data
def get_word_frequencies(text_data):
    # Preprocess the text and then tokenize
    """
    ' '.join(text_data): joins the list of processed text (from multiple reviews) into a single string
    .split(): splits the string into a list of individual words

    Counter(all_words) counts how many times each word appears in the list of words, it returns a dictionary-like object where keys are words, and values are their respective frequencies

    """
    all_words = ' '.join(text_data).split()
    word_freq = Counter(all_words)
    return word_freq


# Function to get word frequencies by rating
def get_word_frequencies_by_rating(df, text_column, rating_column):
    rating_word_freq = {} #this will store word frequencies for each rating
    
    for rating in df[rating_column].unique():
        # Filter reviews for each rating
        filtered_reviews = df[df[rating_column] == rating][text_column]
        
        # Preprocess and get word frequencies for these filtered reviews
        processed_reviews = filtered_reviews.apply(preprocess_text_spacy)
        word_freq = get_word_frequencies(processed_reviews)
        
        rating_word_freq[rating] = word_freq
    
    return rating_word_freq
    

In [14]:
# Get word frequencies for 'pros' and 'cons' grouped by rating
pros_word_freq_by_rating = get_word_frequencies_by_rating(sampled_reviews, 'pros', 'rating')
cons_word_freq_by_rating = get_word_frequencies_by_rating(sampled_reviews, 'cons', 'rating')

# convert the word frequencies into DataFrames for easier visualization
def word_freq_to_dataframe(word_freq_by_rating):
    df = pd.DataFrame()
    
    for rating, freq in word_freq_by_rating.items():
        # Convert the word frequencies into a DataFrame for each rating
        rating_df = pd.DataFrame(freq.items(), columns=["Word", f"Rating_{rating}_Freq"])
        df = pd.merge(df, rating_df, on="Word", how="outer") if not df.empty else rating_df
    
    return df.fillna(0)

# Create DataFrames for pros and cons word frequencies
pros_freq_df = word_freq_to_dataframe(pros_word_freq_by_rating)
cons_freq_df = word_freq_to_dataframe(cons_word_freq_by_rating)

# Combine pros and cons frequencies into a single DataFrame
combined_freq_df = pd.merge(pros_freq_df, cons_freq_df, on="Word", suffixes=('_pros', '_cons'))

# Display the DataFrame with word frequencies
display(combined_freq_df.head())

,Word,Rating_5.0_Freq_pros,Rating_1.0_Freq_pros,Rating_4.0_Freq_pros,Rating_2.0_Freq_pros,Rating_3.0_Freq_pros,Rating_5.0_Freq_cons,Rating_1.0_Freq_cons,Rating_4.0_Freq_cons,Rating_2.0_Freq_cons,Rating_3.0_Freq_cons
0,good,10051.0,3038.0,13610.0,4146.0,10093.0,1584.0,1285.0,1780.0,1166.0,1864.0
1,place,2138.0,381.0,1769.0,422.0,1080.0,547.0,961.0,454.0,487.0,522.0
2,work,12681.0,2721.0,13807.0,3699.0,9362.0,5503.0,6944.0,7551.0,5633.0,7642.0
3,culture,3178.0,153.0,2698.0,222.0,1072.0,306.0,873.0,581.0,820.0,820.0
4,great,10394.0,1299.0,8880.0,2152.0,5372.0,930.0,360.0,726.0,361.0,675.0


In [15]:
pros_freq_df

,Word,Rating_5.0_Freq,Rating_1.0_Freq,Rating_4.0_Freq,Rating_2.0_Freq,Rating_3.0_Freq
0,good,10051.0,3038.0,13610.0,4146.0,10093.0
1,place,2138.0,381.0,1769.0,422.0,1080.0
2,work,12681.0,2721.0,13807.0,3699.0,9362.0
3,culture,3178.0,153.0,2698.0,222.0,1072.0
4,great,10394.0,1299.0,8880.0,2152.0,5372.0
...,...,...,...,...,...,...
14073,hotspot,0.0,0.0,0.0,0.0,1.0
14074,rembersment,0.0,0.0,0.0,0.0,1.0
14075,spitalfied,0.0,0.0,0.0,0.0,1.0
14076,medibuddy,0.0,0.0,0.0,0.0,1.0


In [16]:
"""
This code combines the 'pros' and 'cons' flags for each emotional state by applying a logical OR operation. 
It then drops the 'cons' flags and displays the updated dataframe with the combined emotional flags.
"""

# sort the df in descending order based on any of the rating columns
combined_freq_df.sort_values(by='Rating_5.0_Freq_pros', ascending=False, inplace=True)

# clean column names
combined_freq_df.columns = combined_freq_df.columns.str.replace(r'\.0', '', regex=True)  # remove '.0' from the column names
combined_freq_df.columns = combined_freq_df.columns.str.replace(r'Rating_', '', regex=True)  # remove 'Rating_' from column names

# renames columns from the format 5_Freq_pros to 5_pros (and similarly for cons)
combined_freq_df.columns = combined_freq_df.columns.str.replace(r'(\d+)_Freq_(pros|cons)', r'\1_\2', regex=True)

combined_freq_df.reset_index(drop=True, inplace=True) #reset the index
display(combined_freq_df.head()) #display the df




,Word,5_pros,1_pros,4_pros,2_pros,3_pros,5_cons,1_cons,4_cons,2_cons,3_cons
0,work,12681.0,2721.0,13807.0,3699.0,9362.0,5503.0,6944.0,7551.0,5633.0,7642.0
1,great,10394.0,1299.0,8880.0,2152.0,5372.0,930.0,360.0,726.0,361.0,675.0
2,good,10051.0,3038.0,13610.0,4146.0,10093.0,1584.0,1285.0,1780.0,1166.0,1864.0
3,company,4864.0,1069.0,4079.0,1051.0,2531.0,2065.0,3266.0,2283.0,2273.0,2586.0
4,people,4262.0,1239.0,4875.0,1684.0,3674.0,920.0,2721.0,1561.0,2002.0,2098.0


In [19]:
# sort the df in descending order based on any of the rating columns
pros_freq_df.sort_values(by='Rating_5.0_Freq', ascending=False, inplace=True)
pros_freq_df.columns = pros_freq_df.columns.str.replace(r'Rating_(\d+)\.0_Freq', r'\1_pros', regex=True)

pros_freq_df.reset_index(drop=True, inplace=True) #reset the index
display(pros_freq_df.head()) #display the df

,Word,5_pros,1_pros,4_pros,2_pros,3_pros
0,work,12681.0,2721.0,13807.0,3699.0,9362.0
1,great,10394.0,1299.0,8880.0,2152.0,5372.0
2,good,10051.0,3038.0,13610.0,4146.0,10093.0
3,company,4864.0,1069.0,4079.0,1051.0,2531.0
4,people,4262.0,1239.0,4875.0,1684.0,3674.0


In [20]:
# sort the df in descending order based on any of the rating columns
cons_freq_df.sort_values(by='Rating_5.0_Freq', ascending=False, inplace=True)
cons_freq_df.columns = cons_freq_df.columns.str.replace(r'Rating_(\d+)\.0_Freq', r'\1_cons', regex=True)

cons_freq_df.reset_index(drop=True, inplace=True) #reset the index
display(cons_freq_df.head()) #display the df

,Word,5_cons,1_cons,4_cons,2_cons,3_cons
0,work,5503.0,6944.0,7551.0,5633.0,7642.0
1,hour,2713.0,2226.0,3525.0,2179.0,3438.0
2,con,2712.0,166.0,1057.0,106.0,271.0
3,time,2380.0,2225.0,3198.0,2056.0,2951.0
4,company,2065.0,3266.0,2283.0,2273.0,2586.0


In [21]:
pros_freq_df = pros_freq_df.head(20)
display(pros_freq_df)

cons_freq_df = cons_freq_df.head(20)
cons_freq_df = cons_freq_df.rename(columns={'5_pros': '5_cons'})
display(cons_freq_df)


,Word,5_pros,1_pros,4_pros,2_pros,3_pros
0,work,12681.0,2721.0,13807.0,3699.0,9362.0
1,great,10394.0,1299.0,8880.0,2152.0,5372.0
2,good,10051.0,3038.0,13610.0,4146.0,10093.0
3,company,4864.0,1069.0,4079.0,1051.0,2531.0
4,people,4262.0,1239.0,4875.0,1684.0,3674.0
5,environment,3637.0,280.0,3946.0,617.0,2262.0
6,benefit,3410.0,1012.0,4069.0,1570.0,3309.0
7,culture,3178.0,153.0,2698.0,222.0,1072.0
8,pay,3060.0,1326.0,3552.0,1520.0,3072.0
9,employee,2709.0,665.0,2253.0,794.0,1589.0


,Word,5_cons,1_cons,4_cons,2_cons,3_cons
0,work,5503.0,6944.0,7551.0,5633.0,7642.0
1,hour,2713.0,2226.0,3525.0,2179.0,3438.0
2,con,2712.0,166.0,1057.0,106.0,271.0
3,time,2380.0,2225.0,3198.0,2056.0,2951.0
4,company,2065.0,3266.0,2283.0,2273.0,2586.0
5,long,1694.0,928.0,2204.0,946.0,1622.0
6,good,1584.0,1285.0,1780.0,1166.0,1864.0
7,pay,1378.0,2967.0,2880.0,2741.0,4018.0
8,lot,1324.0,856.0,2043.0,1034.0,2043.0
9,think,1219.0,443.0,437.0,279.0,315.0


In [ ]:
# Function to flag stress, burnout, and anxiety
def emotional_status(text):
    if not isinstance(text, str):  # Check if text is not None or NaN
        return False, False, False, False, False, False  # Return False for all flags if the text is invalid

    text = text.lower()

    # Keywords for flags
    stress_keywords = ['stress', 'overwhelmed', 'nervous', 'pressure', 'stressful', 'deadlines', 'high demand', 'high workload']
    toxic_keywords = ['toxic','negative', 'micromanagement', 'lack of support', 'hostile', 'bullying', 'discrimination']
    burnout_keywords = ['burnout', 'overtasked', 'overworked', 'burned out', 'burning out', 'exhaustion', 'fatigue']
    anxiety_keywords = ['anxious', 'anxiety', 'panic', 'workload stress', 'job insecurity', 'sleepless']
    depression_keywords= ['depressed','sad','no motivation','unmotivated', 'headaches', 'feeling worthless']
    supportive_keywords = ['supportive', 'understanding', 'encouraging', 'teamwork']    
    

    # Remove unwanted phrases
    unwanted_phrases = ['not too stressful', 'no negatives']
    if any(phrase in text for phrase in unwanted_phrases):
        return False, False, False, False, False

    # Check for each set of keywords
    stress_flag = any(keyword in text for keyword in stress_keywords)
    burnout_flag = any(keyword in text for keyword in burnout_keywords)
    anxiety_flag = any(keyword in text for keyword in anxiety_keywords)
    toxic_flag = any(keyword in text for keyword in toxic_keywords)
    supportive_env_flag = any(keyword in text for keyword in supportive_keywords)
    depression_flag = any(keyword in text for keyword in depression_keywords)
    
    return stress_flag, burnout_flag, anxiety_flag, toxic_flag, supportive_env_flag, depression_flag




Save the dataframes for further analysis

In [22]:
#pros_freq_df.to_csv("../data/clean/pros_freq_df.csv")
#cons_freq_df.to_csv("../data/clean/cons_freq_df.csv")

In [23]:
# Count the number of stressed reviews in the 'pros' section (based on stress_flag)
stressed_jobs_pros = sampled_reviews[sampled_reviews['stress_flag'] == True].groupby('job').size()

# Count the number of stressed reviews in the 'cons' section (based on stress_flag_cons)
stressed_jobs_cons = sampled_reviews[sampled_reviews['stress_flag_cons'] == True].groupby('job').size()

# Merge both results into a single DataFrame
stressed_jobs = pd.DataFrame({
    'stress_flag_pros': stressed_jobs_pros,
    'stress_flag_cons': stressed_jobs_cons
}).fillna(0)  # Fill NaN values with 0 for jobs that have no flags

# Sort the results by the combined count
stressed_jobs = stressed_jobs.sort_values(by=['stress_flag_pros', 'stress_flag_cons'], ascending=False)

# Display the results
display(stressed_jobs)


,stress_flag_pros,stress_flag_cons
job,,
Anonymous Employee,230.0,1121.0
Software Engineer,129.0,144.0
Manager,83.0,633.0
Analyst,33.0,198.0
Sales Associate,32.0,152.0
...,...,...
Warehouse Picker,0.0,1.0
Warehouse Supervisor,0.0,1.0
Wealth Management,0.0,1.0


In [24]:
# Convert stressed_jobs to DataFrame
stressed_jobs_df = stressed_jobs.reset_index()

# Limit to first 20 jobs
stressed_jobs_df = stressed_jobs_df.head(20)

In [25]:
stressed_jobs_df

,job,stress_flag_pros,stress_flag_cons
0,Anonymous Employee,230.0,1121.0
1,Software Engineer,129.0,144.0
2,Manager,83.0,633.0
3,Analyst,33.0,198.0
4,Sales Associate,32.0,152.0
5,Consultant,27.0,83.0
6,Team Member,24.0,510.0
7,Associate Analyst,17.0,4.0
8,Cashier,13.0,129.0
9,Associate,13.0,92.0


In [26]:
# Count the number of stressed reviews in the 'pros' section (based on stress_flag)
burned_out_jobs_pros = sampled_reviews[sampled_reviews['stress_flag'] == True].groupby('job').size()

# Count the number of stressed reviews in the 'cons' section (based on stress_flag_cons)
burned_out_jobs_cons = sampled_reviews[sampled_reviews['stress_flag_cons'] == True].groupby('job').size()

# Merge both results into a single DataFrame
stressed_jobs = pd.DataFrame({
    'stress_flag_pros': stressed_jobs_pros,
    'stress_flag_cons': stressed_jobs_cons
}).fillna(0)  # Fill NaN values with 0 for jobs that have no flags

# Sort the results by the combined count
stressed_jobs = stressed_jobs.sort_values(by=['stress_flag_pros', 'stress_flag_cons'], ascending=False)

# Display the results
display(stressed_jobs)

,stress_flag_pros,stress_flag_cons
job,,
Anonymous Employee,230.0,1121.0
Software Engineer,129.0,144.0
Manager,83.0,633.0
Analyst,33.0,198.0
Sales Associate,32.0,152.0
...,...,...
Warehouse Picker,0.0,1.0
Warehouse Supervisor,0.0,1.0
Wealth Management,0.0,1.0


### Combining flags together in order to see the time trends analysis

In [27]:
# List of emotional flags (just the pros flags)
combined_flags = [
    'stress_flag', 'burnout_flag', 'anxiety_flag', 'toxic_flag', 'supportive_env_flag', 'depression_flag'
]

combined_flags_df = sampled_reviews.copy()

# Combine the pros and cons flags for each emotional state
for flag in combined_flags:
    # Combine the 'pros' and 'cons' flags using the logical OR (|)
    combined_flags_df[flag] = combined_flags_df[flag] | combined_flags_df[flag + '_cons']

# Check the dataframe to see if the flags have been combined correctly
display(combined_flags_df[['stress_flag', 'burnout_flag', 'anxiety_flag', 'toxic_flag', 'supportive_env_flag', 'depression_flag']].head())



,stress_flag,burnout_flag,anxiety_flag,toxic_flag,supportive_env_flag,depression_flag
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False


In [28]:
combined_flags_df["job"].value_counts().head(30)

job
Anonymous Employee            26069
Manager                       11040
Team Member                    5214
Software Engineer              4218
Sales Associate                3365
Cashier                        3233
Analyst                        3143
Consultant                     2614
Associate                      1524
Customer Service               1397
Director                       1341
Delivery Driver                 920
Vice President                  898
Shift Leader                    435
Administrative Assistant        421
Software Developer              420
Intern                          387
Game Advisor                    358
Senior Game Advisor             353
Engineer                        340
Barista                         288
Supervisor                      287
HR                              282
Area Supervisor                 273
Warehouse Associate             271
Embedded Software Engineer      270
Account Executive               256
Marketing Associate     

### Aggregation of the years

In [29]:
# Add the 'year' column from the original sampled_reviews dataframe to the combined_flags_df
combined_flags_df['year'] = sampled_reviews['year']

# Group by 'year' and aggregate by summing up the True values (counting True occurrences)
aggregated_flags_by_year = combined_flags_df.groupby('year')[combined_flags].sum()
aggregated_flags_by_year = aggregated_flags_by_year.iloc[:-1] #remove 2023 because it doesnt have enough inputs

# Display the aggregated flags by year
display(aggregated_flags_by_year)


,stress_flag,burnout_flag,anxiety_flag,toxic_flag,supportive_env_flag,depression_flag
year,,,,,,
2015,522,64,25,212,244,31
2016,594,60,19,221,247,49
2017,628,75,18,272,294,56
2018,548,82,15,265,297,51
2019,473,62,12,278,260,43
2020,589,79,25,260,314,40
2021,1118,241,26,534,605,62
2022,1042,223,20,536,668,52


In [30]:
# reset index
aggregated_flags_by_year_reset = aggregated_flags_by_year.reset_index()

# create a plotly line plot
fig = px.line(aggregated_flags_by_year_reset, 
              x='year', 
              y=combined_flags, 
              title='Emotional flags evolution over the years',
              labels={'value': 'Count', 'year': 'Year'},
              markers=True)

# Show the plot
fig.show()


### Save it as csv for further analysis

In [31]:
#aggregated_flags_by_year.to_csv("../data/clean/aggregated_flags_by_year.csv")

### Agregate by job

In [32]:
# Add the 'job' column from the original sampled_reviews dataframe to the combined_flags_df
combined_flags_df['job'] = sampled_reviews['year']

# Group by 'job' and aggregate by summing up the True values (counting True occurrences)
aggregated_flags_by_job = combined_flags_df.groupby('job')[combined_flags].sum()


# calculate the total flags (sum of True values) for each job and sort in descending order
aggregated_flags_by_job['total_flags'] = aggregated_flags_by_job.sum(axis=1)
aggregated_flags_by_job_sorted = aggregated_flags_by_job.sort_values(by='total_flags', ascending=False)

# display the sorted aggregated flags by job
top_20_jobs = aggregated_flags_by_job_sorted.head(20)
top_20_jobs

,stress_flag,burnout_flag,anxiety_flag,toxic_flag,supportive_env_flag,depression_flag,total_flags
job,,,,,,,
2021,1118,241,26,534,605,62,2586
2022,1042,223,20,536,668,52,2541
2017,628,75,18,272,294,56,1343
2020,589,79,25,260,314,40,1307
2018,548,82,15,265,297,51,1258
2016,594,60,19,221,247,49,1190
2019,473,62,12,278,260,43,1128
2015,522,64,25,212,244,31,1098
2023,284,61,7,194,185,12,743


### Saving for further analysis

In [33]:
#top_20_jobs.to_csv("../data/clean/top_20_jobs.csv")

### Agregate flags by companies

In [34]:
aggregated_flags_by_year

,stress_flag,burnout_flag,anxiety_flag,toxic_flag,supportive_env_flag,depression_flag
year,,,,,,
2015,522,64,25,212,244,31
2016,594,60,19,221,247,49
2017,628,75,18,272,294,56
2018,548,82,15,265,297,51
2019,473,62,12,278,260,43
2020,589,79,25,260,314,40
2021,1118,241,26,534,605,62
2022,1042,223,20,536,668,52


In [35]:
# add the 'firm_name' column from the original sampled_reviews dataframe to the combined_flags_df
combined_flags_df['firm_name'] = sampled_reviews['firm_name']

# group by 'firm_name' and aggregate by summing up the True values (counting True occurrences)
aggregated_flags_by_year_company = combined_flags_df.groupby('firm_name')[combined_flags].sum()

# add a column for the total flags
aggregated_flags_by_year_company['total_flags'] = aggregated_flags_by_year_company.sum(axis=1)

# sort by 'total_flags' in descending order
sorted_flags_by_company = aggregated_flags_by_year_company.sort_values(by='total_flags', ascending=False)

#display the top 30 companies
top_20_companies_by_flags = sorted_flags_by_company.head(20)
display(top_20_companies_by_flags)

,stress_flag,burnout_flag,anxiety_flag,toxic_flag,supportive_env_flag,depression_flag,total_flags
firm_name,,,,,,,
Chipotle,932,255,25,172,242,23,1649
Goldman-Sachs,615,65,8,155,233,16,1092
CGI,312,15,3,144,221,12,707
Dunkin,353,98,8,95,128,15,697
GameStop,343,40,12,136,55,22,608
Ross-Stores,220,66,9,85,136,17,533
Valeo,344,6,2,76,86,12,526
H-E-B,173,29,8,89,173,12,484
UBS,181,15,0,104,159,10,469


In [36]:
# reset index
top_20_companies_by_flags = top_20_companies_by_flags.reset_index()

# Create a Plotly line plot
fig = px.bar(top_20_companies_by_flags, 
              x='firm_name', 
              y=combined_flags, 
              title='Emotional status by companies',
              labels={'value': 'Count', 'firm_name': 'Company'})

# Show the plot
fig.show()

### Save the table for further analysis

In [37]:
#top_30_companies_by_flags.to_csv("../data/clean/top_30_companies_by_flags.csv")